In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

Label Read In 

In [2]:
df1 = pd.read_csv('finnum/train.csv')
x = df1.index
df2 = pd.read_csv('finnum/dev.csv')
df = pd.concat([df1,df2], ignore_index = True)
df['cat_num'] = df['category'].astype('category').cat.codes
labels = pd.get_dummies(df['cat_num'][x])
labels2 = pd.get_dummies(df['cat_num'][len(x):])

Loading input matrix

In [3]:
trainInp = np.load('trainChar.npy')
valInp = np.load('valChar.npy')
embed = np.load('embedChar.npy')
embed = embed.astype(np.float32)

In [4]:
learning_rate = 0.001
batch_size = 16

In [5]:
tf.reset_default_graph()

Batching and creating iterators

In [6]:
train = (trainInp, labels)
val = (valInp, labels2)

# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

# create validation Dataset and batch it
val_data = tf.data.Dataset.from_tensor_slices(val)
val_data = val_data.shuffle(10000) # if you want to shuffle your data
val_data = val_data.batch(batch_size)

# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
txt, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
val_init = iterator.make_initializer(val_data)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
embedding = tf.nn.embedding_lookup(embed, txt, partition_strategy='mod', name=None)
embedded_chars_expanded = tf.expand_dims(embedding, -1)

In [8]:
embedding

<tf.Tensor 'embedding_lookup/Identity:0' shape=(?, 547, 8) dtype=float32>

In [9]:
embedded_chars_expanded

<tf.Tensor 'ExpandDims:0' shape=(?, 547, 8, 1) dtype=float32>

Implementing looped convolution

In [13]:
pooled_outputs = []
filter_sizes = [3,4,5]
embedding_size = embed.shape[1]
num_filters1 = 32
num_filters2 = 64
max_length = 547
for filter_size in filter_sizes:
    filter_shape1 = [filter_size, 1, 1, num_filters1]
    W1 = tf.Variable(tf.truncated_normal(filter_shape1, stddev=0.1), name='W1')
    b1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]), name = 'b1')
    conv1 = tf.nn.conv2d(
        embedded_chars_expanded,
        tf.cast(W1,tf.float32),
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv1')
    conv1 = tf.layers.batch_normalization(inputs = conv1)
    relu1 = tf.nn.relu(tf.nn.bias_add(conv1, tf.cast(b1,tf.float32)), name="relu")
    
    filter_shape2 = [filter_size, embedding_size, num_filters1, num_filters2]
    W2 = tf.Variable(tf.truncated_normal(filter_shape2, stddev=0.1), name='W2')
    b2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]), name = 'b2')    
    conv2 = tf.nn.conv2d(
        relu1,
        tf.cast(W2,tf.float32),
        strides=[1, 1, 1, 1],
        padding='VALID',
        name='conv2')
    conv2 = tf.layers.batch_normalization(inputs = conv2)    
    relu2 = tf.nn.relu(tf.nn.bias_add(conv2, tf.cast(b2,tf.float32)), name="relu")
    pooled = tf.nn.max_pool(
        relu2,
        ksize=[1, max_length - filter_size +1, 1, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="pool")
    pooled_outputs.append(pooled)

In [14]:
filter_shape2

[5, 8, 32, 64]

In [15]:
pooled_outputs

[<tf.Tensor 'pool_3:0' shape=(?, 1, 1, 64) dtype=float32>,
 <tf.Tensor 'pool_4:0' shape=(?, 1, 1, 64) dtype=float32>,
 <tf.Tensor 'pool_5:0' shape=(?, 1, 1, 64) dtype=float32>]

Combining separate convolutional layers into 1 feed forward input

In [16]:
num_filters_total = num_filters2 * len(filter_sizes)
combined = tf.concat(pooled_outputs, 3)
combined_flat = tf.reshape(combined, [-1, num_filters_total])

In [17]:
combined_flat

<tf.Tensor 'Reshape:0' shape=(?, 192) dtype=float32>

Adding dense layers

In [18]:
hidden1 = 800
hidden2 = 400
out = 7 

In [19]:
inp_size = int(combined_flat.get_shape()[1])
w_h = tf.get_variable('weight_h', initializer = tf.random_normal([inp_size,hidden1], mean = 0.0, stddev = .01),dtype = tf.float32)
b_h = tf.get_variable('bias_h', initializer = tf.constant(0.0, shape = [1,hidden1]), dtype = tf.float32)

w2 = tf.get_variable('weight2', initializer = tf.random_normal([hidden1,hidden2], mean = 0.0, stddev = .01),dtype = tf.float32)
b2 = tf.get_variable('bias2', initializer = tf.constant(0.0, shape = [1,hidden2]), dtype = tf.float32)

w = tf.get_variable('weight', initializer = tf.random_normal([hidden1,out], mean = 0.0, stddev = .01),dtype = tf.float32)
b = tf.get_variable('bias', initializer = tf.constant(0.0, shape = [1,out]), dtype = tf.float32)

In [20]:
h2 = tf.nn.leaky_relu(tf.matmul(combined_flat, w_h) + b_h)

#h = tf.nn.leaky_relu(tf.matmul(h2, w2) + b2)

#drop2 = tf.nn.dropout(h, keep_prob = .6)
logits = tf.matmul(h2,w) + b

Implementing cross entropy, loss, and optimization

In [21]:
n_epochs = 10
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = label, logits = logits)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Prediction setup

In [22]:
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

Running Neural Net

In [23]:
with tf.Session() as sess:
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())
    # train the model n_epochs times

    for i in range(n_epochs): 
        
        sess.run(train_init)# drawing samples from train_data
        total_loss = 0
        total_right = 0
        n_batches = 0
        totalright = 0
        totalvalright = 0
        nvalbatches = 0
        totalvalloss = 0
        try:
            while True:
                #summary,acc,_, l = sess.run([summary_op,accuracy,optimizer, loss]) #use with scalar summary
                acc,_, l = sess.run([accuracy, optimizer, loss])                
                total_loss += l
                total_right += acc
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        sess.run(val_init)
        try:
            while True:
                accval,valloss = sess.run([accuracy, loss])
                totalvalright += accval
                nvalbatches += 1
                totalvalloss += valloss
        except tf.errors.OutOfRangeError:
            pass

        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        print('Accuracy {0}: {1}'.format(i, total_right/n_batches))    
        print('Average val loss epoch {0}: {1}'.format(i, totalvalloss/nvalbatches))
        print('Val_Accuracy {0}: {1}'.format(i, totalvalright/nvalbatches)) 
    prediction = sess.run(preds, feed_dict={txt: valInp})
    prediction = np.asarray(prediction)

Average loss epoch 0: 1.3265896566744362
Accuracy 0: 0.47619047619047616
Average val loss epoch 0: 1.1730959161799004
Val_Accuracy 0: 0.5771276595744681
Average loss epoch 1: 0.8878861701204663
Accuracy 1: 0.6840773809523809
Average val loss epoch 1: 0.7851193242884696
Val_Accuracy 1: 0.7207446808510638
Average loss epoch 2: 0.7244541338866666
Accuracy 2: 0.7389880952380953
Average val loss epoch 2: 0.7324035446694557
Val_Accuracy 2: 0.7313829787234043
Average loss epoch 3: 0.5933987545825187
Accuracy 3: 0.7879464285714286
Average val loss epoch 3: 0.649250141800718
Val_Accuracy 3: 0.7792553191489362
Average loss epoch 4: 0.5156727180417094
Accuracy 4: 0.8136904761904762
Average val loss epoch 4: 0.6535254152531319
Val_Accuracy 4: 0.7805851063829787
Average loss epoch 5: 0.44603047750535463
Accuracy 5: 0.8398809523809524
Average val loss epoch 5: 0.6237308554192806
Val_Accuracy 5: 0.8111702127659575
Average loss epoch 6: 0.38383904848312905
Accuracy 6: 0.8617559523809524
Average val lo

Calculating accuracy for train

In [24]:
np.mean(np.equal(np.argmax(prediction, 1), labels2.idxmax(axis = 1)))

0.7876344086021505